In [1]:
import numpy as np
from glob import glob
import pandas as pd

files = glob('data/*.csv')
totals = ['Total_PorosityQuantity', 'Total_PorosityQuality', 'Total_UnfilledZones', 'Total_FillingQuality', 'TOTAL_QUALITY']

df_list = list()
target = list()

for i, file in enumerate(files[:100]):
    df = pd.read_csv(file)
    target.append(df.TOTAL_QUALITY.unique()[0])
    df = df.drop(axis=1, columns=totals)  
    df['id'] = i
    
    ts_cols = [col for col in df if col.endswith(('Time', 'id', 'Flow rate', 'Pressure'))]
    df_list.append(df[ts_cols])

df = pd.concat(df_list)
df.head()

Time     Flow rate  Zone2_Pressure  Zone3_Pressure  Zone4_Pressure  \
0  0.065529  6.727330e-07             0.0             0.0             0.0   
1  0.094341  7.170040e-07             0.0             0.0             0.0   
2  0.116504  7.614100e-07             0.0             0.0             0.0   
3  0.165700  8.404510e-07             0.0             0.0             0.0   
4  0.186822  8.817000e-07             0.0             0.0             0.0   

   Zone5_Pressure  Zone6_Pressure  Zone7_Pressure  Zone9_Pressure  \
0             0.0             0.0             0.0             0.0   
1             0.0             0.0             0.0             0.0   
2             0.0             0.0             0.0             0.0   
3             0.0             0.0             0.0             0.0   
4             0.0             0.0             0.0             0.0   

   Zone10_Pressure ...  Zone22_Pressure  Zone23_Pressure  Zone24_Pressure  \
0              0.0 ...              0.0              0.0              0.0   
1              0.0 ...              0.0              0.0              0.0   
2              0.0 ...              0.0              0.0              0.0   
3              0.0 ...              0.0              0.0              0.0   
4              0.0 ...              0.0              0.0              0.0   

   Zone26_Pressure  Zone27_Pressure  Zone28_Pressure  Zone29_Pressure  \
0              0.0              0.0              0.0              0.0   
1              0.0              0.0              0.0              0.0   
2              0.0              0.0              0.0              0.0   
3              0.0              0.0              0.0              0.0   
4              0.0              0.0              0.0              0.0   

   Zone30_Pressure  Zone31_Pressure  id  
0              0.0              0.0   0  
1              0.0              0.0   0  
2              0.0              0.0   0  
3              0.0              0.0   0  
4              0.0              0.0   0  

[5 rows x 31 columns]

In [2]:
df.id.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int64)

In [3]:
type(df.id.values)

numpy.ndarray

In [4]:
from tsfresh import extract_features
from tsfresh.utilities.distribution import MultiprocessingDistributor
distributor = MultiprocessingDistributor(n_workers=8, disable_progressbar=False, progressbar_title="Feature Extraction")
extracted_features = extract_features(df, column_id='id', column_sort='Time', distributor=distributor)
extracted_features.to_csv('temp/extracted_features.csv')

Feature Extraction: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [45:39<00:00, 68.50s/it]


In [ ]:
# Delete NaN/Inf values
extracted_features = extracted_features.dropna(axis=1)
extracted_features = extracted_features.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
        
extracted_features.to_csv('temp/extracted_features_cleaned.csv')

### UTILITIES

In [163]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

seed = 1
folds = 10

def get_classification_report(X, y):
    algorithm = RandomForestClassifier(n_estimators=100, random_state=seed)
    score = cross_val_score(algorithm, X, y, cv=folds).mean()
    print("SCORE: "+ str(score)) 

### READ EXTRACTED FEATURES FROM CSV

In [164]:
import pandas as pd

extracted_features = pd.read_csv('temp/extracted_features_cleaned.csv')
extracted_features.head()

Unnamed: 0  id  Flow rate__abs_energy  Flow rate__absolute_sum_of_changes  \
0           0   0           1.031236e-07                            0.000062   
1           1   1           1.042159e-07                            0.000064   
2           2   2           9.353593e-08                            0.000079   
3           3   3           1.027388e-07                            0.000061   
4           4   4           1.049617e-07                            0.000068   

   Flow rate__agg_autocorrelation__f_agg_"mean"__maxlag_40  \
0                                                0.0         
1                                                0.0         
2                                                0.0         
3                                                0.0         
4                                                0.0         

   Flow rate__agg_autocorrelation__f_agg_"median"__maxlag_40  \
0                                                0.0           
1                                                0.0           
2                                                0.0           
3                                                0.0           
4                                                0.0           

   Flow rate__agg_autocorrelation__f_agg_"var"__maxlag_40  \
0                                                0.0        
1                                                0.0        
2                                                0.0        
3                                                0.0        
4                                                0.0        

   Flow rate__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"intercept"  \
0                                           0.000012                          
1                                           0.000012                          
2                                           0.000013                          
3                                           0.000013                          
4                                           0.000013                          

   Flow rate__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"  \
0                                          -0.243467                       
1                                          -0.253883                       
2                                          -0.410989                       
3                                          -0.381023                       
4                                          -0.428986                       

   Flow rate__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"  \
0                                      -4.506888e-08                      
1                                      -4.662547e-08                      
2                                      -7.622757e-08                      
3                                      -7.325205e-08                      
4                                      -7.536972e-08                      

                            ...                             \
0                           ...                              
1                           ...                              
2                           ...                              
3                           ...                              
4                           ...                              

   Zone9_Pressure__symmetry_looking__r_0.9  \
0                                      1.0   
1                                      1.0   
2                                      1.0   
3                                      1.0   
4                                      1.0   

   Zone9_Pressure__symmetry_looking__r_0.9500000000000001  \
0                                                1.0        
1                                                1.0        
2                                                1.0        
3                                                1.0        
4                                                1.0       

### Generate first model and test

In [165]:
get_classification_report(extracted_features, target)

SCORE: 0.6873737373737374


### Delete features with low variance

In [166]:
# Removing features with low variance
from sklearn.feature_selection import VarianceThreshold

# sel = VarianceThreshold(threshold=0.05)
# extracted_features = pd.DataFrame(sel.fit_transform(extracted_features))
# extracted_features

sel = VarianceThreshold(threshold=0.05)
selector.fit(extracted_features)
columns = extracted_features.columns[selector.get_support(indices=True)]
extracted_features = extracted_features[columns]
extracted_features = extracted_features.loc[:, ~extracted_features.columns.str.contains('^Unnamed')]
extracted_features.drop('id', axis=1, inplace=True)
extracted_features.head()

Flow rate__augmented_dickey_fuller__attr_"teststat"  \
0                                          -0.917134     
1                                          -0.829793     
2                                          -1.476037     
3                                          -0.961903     
4                                          -1.351336     

   Flow rate__augmented_dickey_fuller__attr_"usedlag"  \
0                                                3.0    
1                                                5.0    
2                                                1.0    
3                                                2.0    
4                                                4.0    

   Flow rate__cid_ce__normalize_True  Flow rate__count_above_mean  \
0                           2.027651                        439.0   
1                           2.095977                        439.0   
2                           2.915139                        414.0   
3                           2.037617                        491.0   
4                           2.862844                        528.0   

   Flow rate__count_below_mean  Flow rate__fft_aggregated__aggtype_"centroid"  \
0                        418.0                                      31.771789   
1                        422.0                                      32.024493   
2                        478.0                                      43.982100   
3                        412.0                                      35.702867   
4                        411.0                                      45.995221   

   Flow rate__fft_aggregated__aggtype_"kurtosis"  \
0                                      -5.207888   
1                                      -5.110457   
2                                      -5.713558   
3                                      -5.395782   
4                                      -5.909401   

   Flow rate__fft_aggregated__aggtype_"skew"  \
0                                   3.018917   
1                                   3.015963   
2                                   2.575471   
3                                   2.942215   
4                                   2.586751   

   Flow rate__fft_aggregated__aggtype_"variance"  \
0                                    5994.850747   
1                                    6007.395536   
2                                    8387.874541   
3                                    6980.032859   
4                                    8835.415363   

   Flow rate__fft_coefficient__coeff_10__attr_"angle"  \
0                                         128.443307    
1                                         118.192733    
2                                        -131.016670    
3                                         -90.795881    
4                                          37.502317    

             ...             Zone9_Pressure__sum_of_reoccurring_data_points  \
0            ...                                                        0.0   
1            ...                                                        0.0   
2            ...                                                        0.0   
3            ...                                                        0.0   
4            ...                                                        0.0   

   Zone9_Pressure__sum_of_reoccurring_values  Zone9_Pressure__sum_values  \
0                                        0.0                 6232826.220   
1                                        0.0                 6738437.730   
2                                        0.0                 6841256.860   
3                                        0.0                 8961547.409   
4                                        0.0                 3490432.920   

   Zone9_Pressure__symmetry_looking__r_0.1  \
0                                      0.0   
1                                      1.0   
2                                      1.0   
3                                  

### Delete highly correlated featatures (if 2 features are highly correlated, delete one)

In [167]:
def correl(data, threshold):
    corr = data.corr()
    columns = np.full((corr.shape[0],), True, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i,j] >= threshold:
                if columns[j]:
                    columns[j] = False
    selected_columns = data.columns[columns]
    data = data[selected_columns]
    return data


#def trimm_correlated(df_in, threshold):
#    df_corr = df_in.corr(method='pearson', min_periods=1)
#    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
#     un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
#     df_out = df_in[un_corr_idx]
#     return df_out
# extracted_features = trimm_correlated(extracted_features, 0.95)
extracted_features = correl(extracted_features, 0.9)
extracted_features.head()

Flow rate__augmented_dickey_fuller__attr_"teststat"  \
0                                          -0.917134     
1                                          -0.829793     
2                                          -1.476037     
3                                          -0.961903     
4                                          -1.351336     

   Flow rate__augmented_dickey_fuller__attr_"usedlag"  \
0                                                3.0    
1                                                5.0    
2                                                1.0    
3                                                2.0    
4                                                4.0    

   Flow rate__cid_ce__normalize_True  Flow rate__count_above_mean  \
0                           2.027651                        439.0   
1                           2.095977                        439.0   
2                           2.915139                        414.0   
3                           2.037617                        491.0   
4                           2.862844                        528.0   

   Flow rate__count_below_mean  Flow rate__fft_aggregated__aggtype_"kurtosis"  \
0                        418.0                                      -5.207888   
1                        422.0                                      -5.110457   
2                        478.0                                      -5.713558   
3                        412.0                                      -5.395782   
4                        411.0                                      -5.909401   

   Flow rate__fft_aggregated__aggtype_"skew"  \
0                                   3.018917   
1                                   3.015963   
2                                   2.575471   
3                                   2.942215   
4                                   2.586751   

   Flow rate__fft_coefficient__coeff_10__attr_"angle"  \
0                                         128.443307    
1                                         118.192733    
2                                        -131.016670    
3                                         -90.795881    
4                                          37.502317    

   Flow rate__fft_coefficient__coeff_11__attr_"angle"  \
0                                        -134.379691    
1                                        -127.783891    
2                                        -130.032831    
3                                          26.986556    
4                                        -160.207602    

   Flow rate__fft_coefficient__coeff_12__attr_"angle"  \
0                                         163.932519    
1                                         149.197934    
2                                          -6.859566    
3                                         132.363953    
4                                         104.284825    

                             ...                             \
0                            ...                              
1                            ...                              
2                            ...                              
3                            ...                              
4                            ...                              

   Zone9_Pressure__quantile__q_0.7  \
0                              0.0   
1                              0.0   
2                              0.0   
3                              0.0   
4                              0.0   

   Zone9_Pressure__range_count__max_1__min_-1  \
0                                       691.0   
1                                       705.0   
2                                       700.0   
3                                       692.0   
4                                       747.0   

   Zone9_Pressure__ratio_beyond_r_sigma__r_0.5  \
0                                     0.116686   
1                                     0.106852   
2                                     

In [168]:
get_classification_report(extracted_features, target)

SCORE: 0.6782828282828284


In [169]:
extracted_features .to_csv('temp/extracted_features_cleaned_1.csv')